In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install import-ipynb
%pip install koreanize-matplotlib
%pip install omegaconf
!pip install ultralytics

In [ ]:
import os
import shutil                   # 파일 복사와 같은 고수준 연산 모듈
import tqdm
import json
import yaml
from pathlib import Path

In [ ]:
data_root_dir = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/ai02-level1-project/train/train_annotations'
output_base_dir = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/copy_data/train/annotations'

# 하나의 폴더에 JSON 파일들을 다 넣어줌

In [ ]:
# 1) 출력 폴더(copy_data/train/annotations) 미리 생성
os.makedirs(output_base_dir, exist_ok=True)

# annotations json 파일들을 하나의 폴더에 다 넣어준다.
# 이름이 같은 json이 존재한다. 따라서 이름을 다르게 해줘야한다.
for root, dirs, files in os.walk(data_root_dir):
    for file in files:
        if file.endswith('.json'):              # 확장자가 .json인 파일만
            src_path = os.path.join(root, file)  # 원본 경로
            base, ext = os.path.splitext(file)
            dst_path = os.path.join(output_base_dir, file)  # 목적지 경로

            # 이미 존재하면 _1, _2 이런식으로 붙이기
            counter = 1

            while os.path.exists(dst_path):
              dst_filename = f"{base}_{counter}{ext}"
              dst_path = os.path.join(output_base_dir, dst_filename)
              counter += 1

            shutil.copy(src_path, dst_path)      # 파일 복사

In [ ]:
# data.yaml, labels txt 파일을 class_map
annotations_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/copy_data/train/annotations'
save_dir = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/train/annotations'
class_map = {}
class_map_count = 0

In [ ]:
# cx, cy, w, h 함수
def coco_to_yolo(bbox, img_w, img_h):
    if len(bbox) == 1 and isinstance(bbox[0], (list, tuple)):
        bbox = bbox[0]
    x, y, w, h = bbox
    # 1) 중심 좌표 계산
    cx = x + w / 2
    cy = y + h / 2

    # 2) 정규화
    nx = cx / img_w
    ny = cy / img_h
    nw = w  / img_w
    nh = h  / img_h
    return (nx, ny, nw, nh)

# labels txt 파일

In [ ]:
# labels txt 파일 생성
# annotations_path json 순회
os.makedirs(save_dir, exist_ok=True)

for root, dirs, files in os.walk(annotations_path):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r') as f:
                data = json.load(f)
                file_name = data['images'][0]['imgfile']
                file_name = os.path.splitext(file_name)[0] # (basename, ext) 튜플 반환함
                class_name = data['images'][0]['dl_name']

                # 클래스 이름 클래스 맵에 넣어주기.
                if class_name not in class_map:
                  class_map[class_name] = class_map_count
                  class_map_count += 1

                # YOLO 포맷 수치 계산
                bbox = data['annotations'][0]['bbox']
                img_w = data['images'][0]['width']
                img_h = data['images'][0]['height']
                cx, cy, w, h = coco_to_yolo(bbox, img_w, img_h)

                # txt 파일 경로
                txt_path = os.path.join(save_dir, f'{file_name}.txt')
                mode = 'w'
                if os.path.exists(txt_path):
                  mode = 'a'

                # 쓰기
                with open(txt_path, mode, encoding='utf-8') as f:
                  # 이미 존재하는 파일이면 한줄 띄우기
                  if mode == 'a':
                    f.write('\n')
                  f.write(f'{class_map[class_name]} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}')

# data.yaml

In [ ]:
# data.yaml 파일 생성
yaml_path = '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/data.yaml'
Path(yaml_path).parent.mkdir(parents=True, exist_ok=True)

class_arr = [None] * class_map_count
for key, idx in class_map.items():
  class_arr[idx] = key

data = {
    'train': '/content/drive/Othercomputers/내 MacBook Air/Documents/ai_project/beginner_project/data/train/images',
    'nc': class_map_count,
    'names': class_arr
}

with open(yaml_path, 'w', encoding='utf-8') as f:
  yaml.dump(data, f, allow_unicode=True)